# NordPool Data Analysis

**Introduction**

This is an analysis of data freely available on https://www.nordpoolgroup.com/historical-market-data/.

The NordPool data shows historical market data for electricity spot prices in the Nordics (- Iceland) and the Baltics.

# 1. Importing data

In [2]:
# importing modules

import pandas as pd

# importing 2017-data

elspot_2017 = pd.read_csv("datasets/elspot-prices_2017_hourly_eur.csv", delimiter=";", header=2)

elspot_2017.head(24)

,Unnamed: 0,Hours,SYS,SE1,SE2,SE3,SE4,FI,DK1,DK2,Oslo,Kr.sand,Bergen,Molde,Tr.heim,TromsÃ¸,EE,LV,LT
0,01-01-2017,00 - 01,"25,7","24,03","24,03","24,03","24,03","24,03","20,96","20,96","28,4","28,4","28,4","24,03","24,03","24,03","24,03","24,03","24,03"
1,01-01-2017,01 - 02,"25,61","24,03","24,03","24,03","24,03","24,03","20,9","20,9","28,17","28,17","28,17","24,03","24,03","24,03","24,03","24,03","24,03"
2,01-01-2017,02 - 03,"25,49","24,02","24,02","24,02","24,02","24,02","18,13","18,13","28,03","28,03","28,03","24,02","24,02","24,02","24,02","24,02","24,02"
3,01-01-2017,03 - 04,"25,25","23,19","23,19","23,19","23,19","23,19","16,03","16,03",28,28,28,"23,19","23,19","23,19","23,19","23,19","23,19"
4,01-01-2017,04 - 05,"25,63","24,1","24,1","24,1","24,1","24,1","16,43","16,43","28,02","28,02","28,02","24,1","24,1","24,1","24,1","24,1","24,1"
5,01-01-2017,05 - 06,"25,88","25,01","25,01","25,01","25,01","25,01","13,75","13,75","28,14","28,14","28,14","25,01","25,01","25,01","25,01","25,01","25,01"
6,01-01-2017,06 - 07,"26,58","25,62","25,62","25,62","25,62","25,62","11,1","11,1","28,31","28,31","28,31","25,62","25,62","25,62","25,62","25,62","25,62"
7,01-01-2017,07 - 08,"26,64","25,87","25,87","25,87","25,87","25,87","15,47","15,47","28,09","28,09","28,09","25,87","25,87","25,87","25,87","25,87","25,87"
8,01-01-2017,08 - 09,"26,7","26,18","26,18","26,18","26,18","26,18","16,88","16,88","27,73","27,73","27,73","26,18","26,18","26,18","26,18","26,18","26,18"
9,01-01-2017,09 - 10,"26,89","25,74","25,74","25,74","25,74","25,74","21,81","21,81","28,28","28,28","28,28","25,74","25,74","25,74","25,74","25,74","25,74"
